In [ ]:
import numpy as np

print(np.exp(5))
print(np.exp(1))
print(np.exp(0.1))
ln2=np.log(2)
print(ln2)
ln3=np.log(3)
print(ln3)

print(np.exp(ln2))
print(np.exp(ln3))

148.4131591025766
2.718281828459045
1.1051709180756477
0.6931471805599453
1.0986122886681098
2.0
3.0000000000000004


1.1051709180756477


# 欠席日数の平均値の例 差分プライバシーを満たす

In [ ]:
import numpy as np

sens = 17/6
epsilon = 2
scale = sens/epsilon
DB_X_mean=(1+2+3+10)/4
DB_Xprime_mean=(1+2+3)/3
lap_noise = 1.1677  #X'データベース　　　　ノイズの最小値
lap_noise_2 = 3.1677-DB_X_mean  #Xデータベース　　　　ノイズの最小値

def get_p(lap_noise,scale):
    # Pr[|Y |≧t · b] = exp(−t)　と　 P(kf(ω) = γ)　＝exp(-| γ –f(ω)|/ λ)は同じである
    t = np.abs(lap_noise)/scale
    p = np.exp(-t)

    # x>μ
    if lap_noise>0:
        p = (p*0.5)

    # x<μ
    else:
        p = 1-p*0.5

    print("lap_noise",lap_noise)
    print("scale",scale)
    print("t",t)
    print("p",p)
    print("------------------------------")
    return p

p_X=get_p(lap_noise_2,scale)
p_X_prime=get_p(lap_noise,scale)

result = p_X/p_X_prime
print("最後の確率の比は",result)
print("------------------------------")

lap_noise -0.8323
scale 1.4166666666666667
t 0.5875058823529412
p 0.722144216118809
------------------------------
lap_noise 1.1677
scale 1.4166666666666667
t 0.8242588235294117
p 0.21927996116513163
------------------------------
最後の確率の比は 3.2932522072775674
------------------------------


# 事後信念を求める Posterior belief β(ω) Table３の結果

In [ ]:
import numpy as np

def get_p(query,scale):
    # Pr[|Y |≧t · b] = exp(−t)　と　 P(kf(ω) = γ)　＝exp(-| γ –f(ω)|/ λ)は同じである
    lap_noise=y-query   #データベース{1,2,3}　とし、　ノイズの最小値
    p = 1/(2*scale) * np.exp(-np.abs(lap_noise)/scale)
    
    return p

def get_beta_result(query,sens,epsilon):
    result_list=[]
    for i in range(len(epsilon)):
        scale = sens/epsilon[i]
        p_w1=get_p(query[0],scale) #確率を計算する
        p_w2=get_p(query[1],scale)
        p_w3=get_p(query[2],scale)
        p_w4=get_p(query[3],scale)

        result = p_w1/(p_w1+p_w2+p_w3+p_w4) #事後信念を計算する
        result_list.append(result)

    return result_list

In [ ]:
import pandas as pd

y = 1   # y=0~2までも同じである
sens = 17/6
epsilon = [5, 2, 1, 0.5, 0.1, 0.01]

# possible world={1,2,3}　の場合
query1=(1+2+3)/3
query2=(1+2+10)/3
query3=(2+3+10)/3
query4=(1+3+10)/3
query_mean_list=[query1,query2,query3,query4]
result123=get_beta_result(query_mean_list,sens,epsilon)

# possible world={1,2,10}　の場合
query1=(1+2+10)/3
query2=(1+2+3)/3
query3=(2+3+10)/3
query4=(1+3+10)/3
query_mean_list=[query1,query2,query3,query4]
result1210=get_beta_result(query_mean_list,sens,epsilon)

# possible world={2,3,10}　の場合
query1=(2+3+10)/3
query2=(1+2+3)/3
query3=(1+2+10)/3
query4=(1+3+10)/3
query_mean_list=[query1,query2,query3,query4]
result2310=get_beta_result(query_mean_list,sens,epsilon)

# possible world={1,3,10}　の場合
query1=(1+3+10)/3
query2=(1+2+3)/3
query3=(1+2+10)/3
query4=(2+3+10)/3
query_mean_list=[query1,query2,query3,query4]
result1310=get_beta_result(query_mean_list,sens,epsilon)


# 結果をデータフレーム化する
result = list(zip(result123,result1210,result1310,result2310))
df = pd.DataFrame(result, columns=["{1,2,3}","{1,2,10}","{1,3,10}","{2,3,10}"])
df=df.transpose()
df

,0,1,2,3,4,5
"{1,2,3}",0.970548,0.682519,0.459577,0.347697,0.268050,0.251769
"{1,2,10}",0.015803,0.131464,0.201699,0.230342,0.246859,0.249704
"{1,3,10}",0.008776,0.103901,0.179313,0.217184,0.243972,0.249410
"{2,3,10}",0.004873,0.082117,0.159411,0.204777,0.241119,0.249117


# 事後信念を確認する  β(ω1)の計算

In [ ]:
# p = 1/(2*scale) * np.exp(-np.abs(x)/scale)
import numpy as np

def get_p(query,scale):
    # 確率密度関数を使う
    lap_noise=y-query   #データベース{1,2,3}　とし、　ノイズの最小値
    p = 1/(2*scale) * np.exp(-np.abs(lap_noise)/scale)

    return p

def get_beta_result(query,sens,epsilon):
    result_list=[]
    for i in range(len(epsilon)):
        scale = sens/epsilon[i]
        p_w1=get_p(query[0],scale) #確率を計算する
        p_w2=get_p(query[1],scale)
        p_w3=get_p(query[2],scale)
        p_w4=get_p(query[3],scale)

        if epsilon[i]==2:
            print("p_w1",p_w1)
            print("p_w2",p_w2)
            print("p_w3",p_w3)
            print("p_w4",p_w4)

        result = p_w1/(p_w1+p_w2+p_w3+p_w4) #事後信念を計算する
        result_list.append(result)

    # 結果をデータフレーム化する
    result = list(zip(result_list))
    df = pd.DataFrame(result, columns=["{1,2,3}"])
    df=df.transpose()
    return df

In [ ]:
import pandas as pd

y = 2.202   # y=0~2までも同じである
sens = 17/6
epsilon = [5, 2, 1, 0.5, 0.1, 0.01]

# possible world={1,2,3}　の場合
query1=(1+2+3)/3
query2=(1+2+10)/3
query3=(1+3+10)/3
query4=(2+3+10)/3

query_mean_list=[query1,query2,query3,query4]

df=get_beta_result(query_mean_list,sens,epsilon)
df

p_w1 0.30603919098316856
p_w2 0.07840054006514602
p_w3 0.06196295449195284
p_w4 0.04897170001866708


,0,1,2,3,4,5
"{1,2,3}",0.941704,0.617794,0.424425,0.331708,0.265261,0.2515


# 事後確率の例(学年の平均値)

> インデントされたブロック



In [ ]:
import pandas as pd

y = 2.2013   # y=0~2までも同じである
sens = 5/6
epsilon = [5, 2, 1, 0.5, 0.1, 0.01]

# possible world={1,2,3}　の場合
query1=(1+2+3)/3
query2=(1+2+4)/3
query3=(2+3+4)/3
query4=(1+3+4)/3
query_mean_list=[query1,query2,query3,query4]

df=get_beta_result(query_mean_list,sens,epsilon)
df

p_w1 0.7402269554837518
p_w2 0.8741017920726089
p_w3 0.1764781081923481
p_w4 0.39275925276499263


,0,1,2,3,4,5
"{1,2,3}",0.363887,0.338999,0.302722,0.278189,0.255887,0.250594


# 開示のリスク  Max β(ω1)の計算

In [ ]:
# max_beta= 1/(1+np.exp(-delta_v2/scale)++np.exp(-delta_v3/scale)+np.exp(-delta_v4/scale))
import numpy as np

def get_max_beta(query,sens,epsilon):
    scale = sens/epsilon

    delta_v2=query[1]-query[0]
    delta_v3=query[2]-query[0]
    delta_v4=query[3]-query[0]

    max_beta= 1/(1+np.exp(-delta_v2/scale)+np.exp(-delta_v3/scale)+np.exp(-delta_v4/scale))

    return max_beta

In [ ]:
import pandas as pd

y = 1  # y=0~2までも同じである
sens = 5/6
epsilon = 0.5

# possible world={1,2,3}　の場合
query1=(1+2+3)/3
query2=(1+2+4)/3
query3=(2+3+4)/3
query4=(1+3+4)/3

query_mean_list=[query1,query2,query3,query4]

max_beta = get_max_beta(query_mean_list,sens,epsilon)
print(max_beta)

0.32917882930128367


In [ ]:
kf2=np.exp(-1/5)
kf3=np.exp(-2/5)
kf4=np.exp(-3/5)
result = 1/(1+kf2+kf3+kf4)
print(result)

0.6703200460356393
0.3291788293012836


# 二分探索を実行してε の値を検索する

In [ ]:
import numpy as np

def get_max_beta(query,sens,epsilon):
    scale = sens/epsilon

    delta_v2=query[1]-query[0]
    delta_v3=query[2]-query[0]
    delta_v4=query[3]-query[0]

    max_beta= 1/(1+np.exp(-delta_v2/scale)+np.exp(-delta_v3/scale)+np.exp(-delta_v4/scale))

    return max_beta

def search_epsilon():
    epsilon_s = 0
    epsilon_f = 10 #任意のepsilonを設定した
    delta=1/3

    while epsilon_s < epsilon_f:
        epsilon = (epsilon_f + epsilon_s) / 2
        max_beta = get_max_beta(query_mean_list,sens,epsilon)
        
        if max_beta < delta:
            epsilon_s = epsilon
            # print("epsilon_s",epsilon_s)

        elif max_beta > delta:
            epsilon_f = epsilon
            # print("最大限",epsilon_f)
        else:
            return epsilon
    else:
        return epsilon    

In [ ]:
import pandas as pd

y = 1  
sens = 17/6

# possible world={1,2,3}　の場合
query1=(1+2+3)/3
query2=(1+2+10)/3
query3=(2+3+10)/3
query4=(1+3+10)/3

query_mean_list=[query1,query2,query3,query4]

print("search_epsilon",search_epsilon())

search_epsilon 0.43172011956720147


In [ ]:
y = 1  
sens = 5/6

# possible world={1,2,3}　の場合
query1=(1+2+3)/3
query2=(1+2+4)/3
query3=(2+3+4)/3
query4=(1+3+4)/3
query_mean_list=[query1,query2,query3,query4]

print("search_epsilon",search_epsilon())

search_epsilon 0.5251496872695116


# table 2の結果を計算する (まだ)　中間値の例かも

In [ ]:
import numpy as np

def get_p(query,scale):
    # Pr[|Y |≧t · b] = exp(−t)　と　 P(kf(ω) = γ)　＝exp(-| γ –f(ω)|/ λ)は同じである
    lap_noise=y-query   #データベース{1,2,3}　とし、　ノイズの最小値
    p = 1/(2*scale) * np.exp(-np.abs(lap_noise)/scale)
    
    return p

def get_beta_result(query,sens,epsilon):
    result_list=[]
    for i in range(len(epsilon)):
        scale = sens/epsilon[i]
        p_w1=get_p(query[0],scale) #確率を計算する
        p_w2=get_p(query[1],scale)
        p_w3=get_p(query[2],scale)
        p_w4=get_p(query[3],scale)

        result = p_w1/(p_w1+p_w2+p_w3+p_w4) #事後信念を計算する
        result_list.append(result)

    return result_list

In [ ]:
import pandas as pd

y = 1    # y=0~2までも同じである
sens = 5/6
epsilon = [5, 2, 1, 0.5, 0.1, 0.01]

# possible world={1,2,3}　の場合
query1=(1+2+3)/3
query2=(1+2+4)/3
query3=(2+3+4)/3
query4=(1+3+4)/3
query_mean_list=[query1,query2,query3,query4]
result123=get_beta_result(query_mean_list,sens,epsilon)

# possible world={1,2,10}　の場合
query1=(1+2+4)/3
query2=(1+2+3)/3
query3=(2+3+4)/3
query4=(1+3+4)/3
query_mean_list=[query1,query2,query3,query4]
result1210=get_beta_result(query_mean_list,sens,epsilon)

# possible world={2,3,10}　の場合
query1=(2+3+4)/3
query2=(1+2+3)/3
query3=(1+2+4)/3
query4=(1+3+4)/3
query_mean_list=[query1,query2,query3,query4]
result2310=get_beta_result(query_mean_list,sens,epsilon)

# possible world={1,3,10}　の場合
query1=(1+3+4)/3
query2=(1+2+3)/3
query3=(1+2+4)/3
query4=(2+3+4)/3
query_mean_list=[query1,query2,query3,query4]
result1310=get_beta_result(query_mean_list,sens,epsilon)

# 結果をデータフレーム化する
result = list(zip(result123,result1210,result1310,result2310))
df = pd.DataFrame(result, columns=["{1,2,3}","{1,2,10}","{1,3,10}","{2,3,10}"])
df=df.transpose()
df

,0,1,2,3,4,5
"{1,2,3}",0.864955,0.574071,0.413079,0.329179,0.265194,0.251502
"{1,2,10}",0.117059,0.257947,0.276895,0.269509,0.254795,0.250498
"{1,3,10}",0.015842,0.115903,0.185608,0.220655,0.244805,0.249498
"{2,3,10}",0.002144,0.052079,0.124417,0.180657,0.235206,0.248502


In [ ]:
import numpy as np

def get_p(query,scale):
    # Pr[|Y |≧t · b] = exp(−t)　と　 P(kf(ω) = γ)　＝exp(-| γ –f(ω)|/ λ)は同じである
    lap_noise=y-query   #データベース{1,2,3}　とし、　ノイズの最小値
    p = 1/(2*scale) * np.exp(-np.abs(lap_noise)/scale)
    
    return p

def get_beta_result(query,sens,epsilon):
    result_list=[]
    for i in range(len(epsilon)):
        scale = sens/epsilon[i]
        p_w1=get_p(query[0],scale) #確率を計算する
        p_w2=get_p(query[1],scale)
        p_w3=get_p(query[2],scale)
        p_w4=get_p(query[3],scale)

        result = p_w1/(p_w1+p_w2+p_w3+p_w4) #事後信念を計算する
        result_list.append(result)

    return result_list

In [ ]:
import pandas as pd

y = 3.16   # y=0~2までも同じである
sens = 4
epsilon = [5, 2, 1, 0.5, 0.1, 0.01]

q1=[1,2,3]
q2=[1,2,10]
q3=[1,3,10]
q4=[2,3,10]
m1 = np.median(q1)
m2 = np.median(q2)
m3 = np.median(q3)
m4 = np.median(q4)

# {1,2,3}　の場合
query_median_list=[m1,m2,m3,m4]
result123=get_beta_result(query_median_list,sens,epsilon)

# {1,2,10}　の場合
query_median_list=[m2,m1,m3,m4]
result1210=get_beta_result(query_median_list,sens,epsilon)

# possible world={1,3,10}　の場合
query_median_list=[m3,m2,m1,m4]
result1310=get_beta_result(query_median_list,sens,epsilon)

# possible world={2,3,10}　の場合
query_median_list=[m4,m1,m2,m3]
result2310=get_beta_result(query_median_list,sens,epsilon)

# 結果をデータフレーム化する
result = list(zip(result123,result1210,result1310,result2310))
df = pd.DataFrame(result, columns=["{1,2,3}","{1,2,10}","{1,3,10}","{2,3,10}"])
df=df.transpose()
df

,0,1,2,3,4,5
"{1,2,3}",0.11135,0.18877,0.218912,0.234395,0.246875,0.249688
"{1,2,10}",0.11135,0.18877,0.218912,0.234395,0.246875,0.249688
"{1,3,10}",0.38865,0.31123,0.281088,0.265605,0.253125,0.250312
"{2,3,10}",0.38865,0.31123,0.281088,0.265605,0.253125,0.250312
